In [1]:
from repository.cassandra_provider import CassandraProvider
from ml.genre_classifier import GenreClassifier
from ml.views_predictor import ViewsPredictor


In [ ]:
from spark.spark_config import create_spark_session
spark = create_spark_session()


In [3]:
cassandra_provider = CassandraProvider()
df = cassandra_provider.load(spark)


In [4]:
train, test = df.randomSplit(weights=[0.8, 0.2], seed=200)


In [ ]:
gr = GenreClassifier()

gr_results = gr.fit(train).predict(test)
gr_evaluation = gr.evaluate_all(gr_results)

In [6]:
print(gr_evaluation)

   accuracy        f1  weightedPrecision  weightedRecall
0  0.818106  0.787121           0.776236        0.816539


In [7]:
gr_results.sort("views", ascending=False).limit(10).toPandas()


,id,artist,tag,title,views,word_vectors,tag_index,features,rawPrediction,probability,prediction
0,553931,Adam Mickiewicz,misc,Pan Tadeusz - Inwokacja,1865798,"[0.0565991740503038, -0.03586697340399648, 0.0...",3.0,"[0.0565991740503038, -0.03586697340399648, 0.0...","[1.2656372252907122, 2.7484715675005686, 1.968...","[0.10623969270031619, 0.4680298778451104, 0.21...",1.0
1,2029397,Taco Hemingway,rap,6 zer,344414,"[-0.05327897261055868, -0.10824612087663515, 0...",0.0,"[-0.05327897261055868, -0.10824612087663515, 0...","[3.3482671877425916, 1.0252525603323064, 0.691...","[0.8151581227759856, 0.07986737789167211, 0.05...",0.0
2,5092219,Mata,rap,Patointeligencja,310719,"[-0.05081893871531631, -0.06867769900850296, 0...",0.0,"[-0.05081893871531631, -0.06867769900850296, 0...","[5.291366305608189, -0.15872174671133554, -0.2...","[0.9789980707169285, 0.0042057036651913455, 0....",0.0
3,327365,Eripe x Quebonafide,rap,Prowo,304558,"[-0.028140135611031663, -0.014489627847644059,...",0.0,"[-0.028140135611031663, -0.014489627847644059,...","[5.596174877666104, -0.0625027010357837, -0.39...","[0.985951701525443, 0.0034381371221600208, 0.0...",0.0
4,5188731,Paris Platynov,rap,HOTELOWY FREESTYLE,289676,"[-0.08874175558935518, -0.0488890867041038, 0....",0.0,"[-0.08874175558935518, -0.0488890867041038, 0....","[5.446539882157381, 0.18519780849697076, -0.65...","[0.9799402476821317, 0.005084260160351864, 0.0...",0.0
5,3058392,Quebonafide,rap,Cest la Vie,231780,"[0.1330781336308869, 0.06976153864219907, 0.22...",0.0,"[0.1330781336308869, 0.06976153864219907, 0.22...","[2.8063580486691966, 3.1427650663401607, -0.76...","[0.39527953465177645, 0.5533552580865629, 0.01...",1.0
6,2810938,Taco Hemingway,rap,Deszcz na betonie,215613,"[0.01652138757002403, -0.11031870971406427, 0....",0.0,"[0.01652138757002403, -0.11031870971406427, 0....","[2.2968452736894354, 2.5760566033276935, 1.221...","[0.3602531529769848, 0.4762859059505804, 0.122...",1.0
7,3376425,Bedoes & Kubi Producent,rap,05:05,211734,"[-0.0283162603013669, -0.08440886317949856, 0....",0.0,"[-0.0283162603013669, -0.08440886317949856, 0....","[3.3435674230543535, 1.0215490924977684, 0.416...","[0.8271229030403078, 0.08112044021431883, 0.04...",0.0
8,3065829,Quebonafide,rap,Zorza,195184,"[0.0034886569649513277, -0.07163304948068645, ...",0.0,"[0.0034886569649513277, -0.07163304948068645, ...","[4.647737891425058, 0.49336405953300755, 0.448...","[0.9572043068531132, 0.015023911375109633, 0.0...",0.0
9,2470767,PRO8L3M,rap,Molly,193069,"[-0.02478589512311559, -0.051998302226468014, ...",0.0,"[-0.02478589512311559, -0.051998302226468014, ...","[5.129598695106024, 0.5571918254987642, -0.167...","[0.9730195208719461, 0.010054268542590476, 0.0...",0.0


In [ ]:
vp = ViewsPredictor()

train_mapped, test_mapped = vp.logarithmise_label_and_get_avg_views(train, test)

vp_results = vp.fit(train_mapped).predict(test_mapped)
vp_evaluation = vp.evaluate_all(vp_results)


In [9]:
print(vp_evaluation)

      rmse        r2
0  1.13534  0.722831


In [10]:
vp_results.sort("views", ascending=False).limit(10).toPandas()


,artist,id,tag,title,views,word_vectors,views_log,artist_avg_views,tag_index,features,prediction
0,Adam Mickiewicz,553931,misc,Pan Tadeusz - Inwokacja,1865798,"[0.0565991740503038, -0.03586697340399648, 0.0...",14.439200,4.731033,3.0,"[0.0565991740503038, -0.03586697340399648, 0.0...",4.177779
1,Quebonafide,5240143,rap,ROMANTICPSYCHO,239558,"[-0.05345424741233782, -0.08227868943623746, 0...",12.386555,10.144769,0.0,"[-0.05345424741233782, -0.08227868943623746, 0...",9.204361
2,Quebonafide,5411320,rap,BUBBLETEA,229619,"[0.016639173674985984, -0.08069375495417698, 0...",12.344181,10.144769,0.0,"[0.016639173674985984, -0.08069375495417698, 0...",9.405644
3,Mata,5092300,rap,GOMBAO 33,225523,"[0.015008921474440112, -0.2691422674934507, 0....",12.326182,10.265764,0.0,"[0.015008921474440112, -0.2691422674934507, 0....",8.460286
4,Taco Hemingway,3178722,rap,Nostalgia,216403,"[-0.004972895757796673, -0.11748052154793047, ...",12.284902,10.947085,0.0,"[-0.004972895757796673, -0.11748052154793047, ...",8.975704
5,ReTo,3700075,rap,UA,215266,"[-0.07767287366811863, -0.08698837589133869, 0...",12.279634,8.774824,0.0,"[-0.07767287366811863, -0.08698837589133869, 0...",8.857798
6,TACONAFIDE,3585011,rap,Art-B,213712,"[0.006288606781350159, 0.016805721306247745, -...",12.272389,10.759390,0.0,"[0.006288606781350159, 0.016805721306247745, -...",9.146779
7,Quebonafide,3105552,rap,Kawa i xanax,204835,"[-0.04548617502884336, -0.1311005343999517, 0....",12.229965,10.144769,0.0,"[-0.04548617502884336, -0.1311005343999517, 0....",9.811915
8,Taco Hemingway,4703918,rap,KABRIOLETY,203205,"[-0.02833912819880611, -0.07430131285350292, 0...",12.221976,10.947085,0.0,"[-0.02833912819880611, -0.07430131285350292, 0...",9.316507
9,"Malik Montana, DaChoyce & SRNO",7644235,rap,Jetlag,198135,"[-0.043330651176259366, 0.0029531332764969167,...",12.196709,12.196709,0.0,"[-0.043330651176259366, 0.0029531332764969167,...",9.307505


In [11]:
spark.stop()